<a href="https://colab.research.google.com/github/khanhtran94/LearnConda/blob/master/deeplearning_15_nn_classifcation_visualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#import the dependencies
import cv2
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf


In [0]:
tf.reset_default_graph()

In [0]:
from tensorflow.examples.tutorials.mnist import input_data
mnist_data = input_data.read_data_sets("./data", one_hot = True)

In [4]:
num_training = mnist_data.train.num_examples
num_testing = mnist_data.test.num_examples
num_validation = mnist_data.validation.num_examples
print("MNIST datasize: Traninning samples: {0}, Testing samples: {1}, Validation sample: {2}".format(num_training,num_testing,num_validation))

MNIST datasize: Traninning samples: 55000, Testing samples: 10000, Validation sample: 5000


In [0]:
# network parameter of neural network
n_input = 784 # input image of size 28 x 28
n_hidden_1 = 512 # first hidden layer
n_hidden_2 = 256 # second hidden layer
n_hidden_3 = 128 # third hidden layer
n_output = 10 # output layer having ( 0 - 9 ) digits

In [0]:
# hyperparameterc for nn
learning_rate = 1e-4
epochs = 3000
batch_size = 128
keep_prob = tf.placeholder(tf.float32)

In [0]:
X = tf.placeholder(tf.float32, [None, n_input])
Y = tf.placeholder(tf.float32, [None, n_output])

In [8]:
with tf.name_scope("Network_parameters") as scope:
  nn_weight = {"W1": tf.Variable(tf.truncated_normal([n_input, n_hidden_1], stddev = 0.1)),
             "W2": tf.Variable(tf.truncated_normal([n_hidden_1, n_hidden_2], stddev = 0.1)),
             "W3": tf.Variable(tf.truncated_normal([n_hidden_2, n_hidden_3], stddev = 0.1)),
             "Wout": tf.Variable(tf.truncated_normal([n_hidden_3, n_output], stddev = 0.1))
            }
  nn_bias = {"B1": tf.Variable(tf.truncated_normal([n_hidden_1])),
           "B2": tf.Variable(tf.truncated_normal([n_hidden_2])),
           "B3": tf.Variable(tf.truncated_normal([n_hidden_3])),
           "B4": tf.Variable(tf.truncated_normal([n_output])),        
          }


Instructions for updating:
Colocations handled automatically by placer.


In [9]:
with tf.name_scope("Network_Model") as scope:
  nn_layer_1 = tf.add(tf.matmul(X, nn_weight["W1"]), nn_bias["B1"])
  nn_layer_2 = tf.add(tf.matmul(nn_layer_1, nn_weight["W2"]), nn_bias["B2"])
  nn_layer_3 = tf.add(tf.matmul(nn_layer_2, nn_weight["W3"]), nn_bias["B3"])

  layer_drop = tf.nn.dropout(nn_layer_3, keep_prob)
  output_layer = tf.add(tf.matmul(layer_drop, nn_weight["Wout"]), nn_bias["B4"])

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
# summary report for weight of neural network
W1_histogram = tf.summary.histogram("W1", nn_weight["W1"])
W2_histogram = tf.summary.histogram("W2", nn_weight["W2"])
W3_histogram = tf.summary.histogram("W3", nn_weight["W3"])
W4_histogram = tf.summary.histogram("W4", nn_weight["Wout"])

# summary report for bias of neural network

B1_histogram = tf.summary.histogram("B1", nn_bias["B1"])
B2_histogram = tf.summary.histogram("B2", nn_bias["B2"])
B3_histogram = tf.summary.histogram("B3", nn_bias["B3"])
B4_histogram = tf.summary.histogram("B4", nn_bias["B4"])


In [0]:
#define the loss
with tf.name_scope("Cost_function") as scope:
  computer_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = output_layer, labels = Y))

#summary for the cost
network_cost = tf.summary.scalar("Cost", computer_loss)

with tf.name_scope("Network_Optimizer") as scope:
  optimizer = tf.train.GradientDescentOptimizer(learning_rate = learning_rate).minimize(computer_loss)

with tf.name_scope("Network_prediction") as scope:
  prediction_out = tf.equal(tf.argmax(output_layer,1), tf.argmax(Y,1))
with tf.name_scope("Network_Accuracy") as scope:
  nn_accuracy = tf.reduce_mean(tf.cast(prediction_out, tf.float32))

init = tf.global_variables_initializer()

merged_summary = tf.summary.merge_all()

saver = tf.train.Saver()

In [12]:
with tf.Session() as sess:
  sess.run(init)
  writer = tf.summary.FileWriter("./log", sess.graph)
  
  for i in range(epochs):
    mini_batch_x, mini_batch_y = mnist_data.train.next_batch(batch_size)
    #print(mini_batch_x[0:1,:].shape)
    mini_batch_val_x, mini_batch_val_y = mnist_data.validation.next_batch(batch_size)
    
    sess.run(optimizer, feed_dict = {X: mini_batch_x, Y:mini_batch_y, keep_prob:1.0})
    
    #running the summary
    summary_epoch = sess.run(merged_summary, feed_dict ={X: mini_batch_x, Y: mini_batch_y, keep_prob:1.0})
    writer.add_summary(summary_epoch,i)
    
    if i%100 == 0:
      mini_batch_loss, mini_batch_accuracy = sess.run([computer_loss, nn_accuracy], feed_dict = {X: mini_batch_x, Y: mini_batch_y, keep_prob:0.85})
      mini_batch_val_loss, mini_batch_val_accuracy = sess.run([computer_loss, nn_accuracy], feed_dict = {X: mini_batch_val_x, Y: mini_batch_val_y, keep_prob:0.85})

      print("Iterations: {0}, Train_loss = {1}, Train_accuracy: {2}, val_accuracy {4}".format(i,mini_batch_loss,mini_batch_accuracy,mini_batch_val_loss,mini_batch_val_accuracy))
  print("Optimization finished")
  test_accuracy = sess.run(nn_accuracy, feed_dict = {X: mnist_data.test.images, Y: mnist_data.test.labels, keep_prob:0.85})
  print("Testing accuracy is {0}".format(test_accuracy))
    
  save_path = saver.save(sess, "./model/my_model.ckpt")

Iterations: 0, Train_loss = 5.658945083618164, Train_accuracy: 0.15625, val_accuracy 0.09375
Iterations: 100, Train_loss = 3.940028667449951, Train_accuracy: 0.140625, val_accuracy 0.1171875
Iterations: 200, Train_loss = 3.2335364818573, Train_accuracy: 0.1640625, val_accuracy 0.15625
Iterations: 300, Train_loss = 3.05216121673584, Train_accuracy: 0.21875, val_accuracy 0.2265625
Iterations: 400, Train_loss = 2.772552967071533, Train_accuracy: 0.265625, val_accuracy 0.2578125
Iterations: 500, Train_loss = 2.919818878173828, Train_accuracy: 0.1953125, val_accuracy 0.3203125
Iterations: 600, Train_loss = 2.42767333984375, Train_accuracy: 0.3046875, val_accuracy 0.296875
Iterations: 700, Train_loss = 2.3484888076782227, Train_accuracy: 0.3125, val_accuracy 0.328125
Iterations: 800, Train_loss = 2.3190507888793945, Train_accuracy: 0.390625, val_accuracy 0.3984375
Iterations: 900, Train_loss = 2.071589946746826, Train_accuracy: 0.4375, val_accuracy 0.3515625
Iterations: 1000, Train_loss = 2.

In [13]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip

--2019-03-01 15:33:20--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.207.111.186, 52.203.66.95, 52.3.53.115, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.207.111.186|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5363700 (5.1M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.1’

ngrok-stable-linux- 100%[===================>]   5.11M  7.73MB/s    in 0.7s    

2019-03-01 15:33:21 (7.73 MB/s) - ‘ngrok-stable-linux-amd64.zip.1’ saved [5363700/5363700]



In [14]:
!unzip ngrok-stable-linux-amd64.zip

Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ngrok                   


In [0]:
LOG_DIR = './log'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

get_ipython().system_raw('./ngrok http 6006 &')

In [16]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

http://551f3e4b.ngrok.io


In [0]:
tf.reset_default_graph()